## 작동 수도 코드

서버 작동 방식
- t 분마다 한 번씩 현 시점에서 n초 이후의 온도를 예측하고 결과(pred_temp)를 받아옴
- 알고리즘의 판단에 따라 작동 변수를 조정한 뒤 조정된 정보를 서버에 넘겨 값을 업데이트 함

사용 변수
- 현재 온도: curr_temp
- 예측 온도: pred_temp
- 현재 환기온도(pre-defined): max_thres
- 현재 난방온도(pre-defined): min_thres
- 예측 시점 - 현재 시점: n


작동 변수(상호 반대)
- 음의 방향 작용(온도를 내리는)
  * 차광스크린, 차열스크린, 에너지스크린(수평, 수직), 배기팬, 천창, (유동팬) 값 증가
- 양의 방향으로 작용(온도를 올리는)
  * 팬코일 값 증가

***

### 1. 경계: 최소 온도와 최대 온도

변수 설명
- 실제 서버에서 주기적으로 가져옴
- 작동 결과를 서버에 업데이트 함

In [1]:
# 현재 온도는 25도, n초 후 예측된 온도는 35도, 현 시점에서 최대 온도(환기온도)는 30도, 최소 온도(난방온도)는 18도
curr_temp = 25
pred_temp = 35
max_thres = 30
min_thres = 18

고민 거리: 전부 다 하이퍼 파라미터로 때리기로 함!
- 이거 넣을지 말지 결정
  * RRS = 0.7 # 변동에 너무 심하게 변하지 않게 쿠션감을 주는 값이라고 함! ex) 10만큼 변화해야하면 7만큼만 변화를 주는 것
- 스크린이 0에서 100으로 바로 증가하는 게 아니라 증가 term이 있을 텐데, 이걸 어떻게 처리해야할지
  * 데이터 보니까 대부분 바로 했던데 우리도 그냥 해도 될듯

In [2]:
# 양의 효과
def effect_plus():
    """
    배기팬, 유동팬, 천창: 0(정지)
    팬코일: 1(작동)
    차광스크린: 0%(감소)
    차열스크린: 0%(감소)
    에너지스크린(수평): 0%(감소)
    에너지스크린(수직): 0%(감소)

    """
    print("작동: 양의 방향")
    print("------------------------")
    return [0, 0, 0, 1, 0, 0, 0, 0]

# 음의 효과
def effect_minus():
    """
    배기팬, 유동팬, 천창: 1(작동)
    팬코일: 0(정지)
    차광스크린: 100%(증가)
    차열스크린: 100%(증가)
    에너지스크린(수평): 100%(증가)
    에너지스크린(수직): 100%(증가)

    """
    print("작동: 음의 방향")
    print("------------------------")
    return [1, 1, 1, 0, 100, 100, 100, 100]
    

In [3]:
# 최대-최소에 의한 작동
def manipulate_minmax(curr_temps, pred_temps, max_thres, min_thres, RRS):
    # 온도 차
    temp_diff = pred_temp - curr_temps
    
    # RRS 적용 시 
    ## temp_diff*=RRS
    
    # 최소 온도 > 예측 온도 : 상승
    if (min_thres > pred_temp):
        print("현재 온도: %d\n예측 온도: %d\n최대 온도: %d\n최소 온도: %d"%(curr_temps, pred_temp, max_thres, min_thres))
        return effect_plus()
    
    # 최대 온도 > 예측 온도 > 최소 온도: 유지
    elif (max_thres > pred_temp) and (pred_temp > min_thres):
        print("현재 온도: %d\n예측 온도: %d\n최대 온도: %d\n최소 온도: %d"%(curr_temps, pred_temp, max_thres, min_thres))
        print("작동 정지: 온도가 적정 범위 내에 있음")
        print("------------------------")
        return [-1, -1, -1, -1, -1, -1, -1, -1 ]

    # 예측 온도 > 최대 온도: 하강
    elif (pred_temp > max_thres):
        print("현재 온도: %d\n예측 온도: %d\n최대 온도: %d\n최소 온도: %d"%(curr_temps, pred_temp, max_thres, min_thres))
        return effect_minus()

    # elif 에러에러 하나 있음, 처리 필요
    else: 
        print("현재 온도: %d\n예측 온도: %d\n최대 온도: %d\n최소 온도: %d"%(curr_temps, pred_temp, max_thres, min_thres))
        print("에러에러")
        return [-1, -1, -1, -1, -1, -1, -1, -1 ]

In [ ]:
# 최적값에 의한 작동
def manipulate_optval(opt_temps, pred_temps, RRS):
    # 온도 차
    temp_diff = pred_temp - opt_temps
    
    # RRS 적용 시 
    ## temp_diff*=RRS
    
    # 최적 온도 > 예측 온도: 상승
    if (temp_diff < 0):
      print("최적 온도: %d\n예측 온도: %d\n온도 차: %d"%(opt_temps, pred_temp, temp_diff))
      return effect_plus()
    
    # 예측 온도 > 최적 온도: 하강
    elif (temp_diff > 0):
      print("최적 온도: %d\n예측 온도: %d\n온도 차: %d"%(opt_temps, pred_temp, temp_diff))
      return effect_minus()
    
    else:
      print("규격 외")
      return [-1, -1, -1, -1, -1, -1, -1, -1 ]

In [4]:
# 예시
curr_temps = [24, 26, 25, 23]
pred_temps = [30, 31, 24, 18]
max_thres = [29, 32, 29, 28]
min_thres = [20, 22, 19, 19]

RRS = 0.7

for pred_temp in pred_temps:
    print("="*30)
    
    # 동일 시점
    idx=pred_temps.index(pred_temp)
    
    # 온도 차
    temp_diff = pred_temp - curr_temps[idx]
    
    # RRS 적용 시 
    ## temp_diff*=RRS

    manipulate_minmax(curr_temps[idx], pred_temp, max_thres[idx], min_thres[idx], RRS)
    

현재 온도: 24
예측 온도: 30
최대 온도: 29
최소 온도: 20
작동: 음의 방향
------------------------
현재 온도: 26
예측 온도: 31
최대 온도: 32
최소 온도: 22
작동 정지: 온도가 적정 범위 내에 있음
------------------------
현재 온도: 25
예측 온도: 24
최대 온도: 29
최소 온도: 19
작동 정지: 온도가 적정 범위 내에 있음
------------------------
현재 온도: 23
예측 온도: 18
최대 온도: 28
최소 온도: 19
작동: 양의 방향
------------------------


In [5]:
import pandas as pd
from keras.models import Sequential, load_model


model = load_model('../Bidirectional_LSTM_Multivariate.h5')
df=pd.read_csv("../Data/smart_farm_test.csv")

In [6]:
df=df[:10]
df.head()

,시간,년월일,년도,월,일,시,분,일사량(W/㎡),외기온(℃),외기습도(%),...,차열스크린(수평),에너지스크린(수평),에너지스크린(수직),배기팬,천창,유동팬,팬코일 B동,환기온도(천창 제어온도),난방온도(설정온도),열공급량(kWh)
0,2022-03-04 00:00:00,2022-03-04 00:00:00,2022,3,4,0,0,-4.0,2.0,76,...,0,0,0,0,0,1,1.0,28.0,18.0,NaN
1,2022-03-04 00:01:00,2022-03-04 00:00:00,2022,3,4,0,1,-4.1,2.0,76,...,0,0,0,0,0,1,1.0,28.0,18.0,NaN
2,2022-03-04 00:02:00,2022-03-04 00:00:00,2022,3,4,0,2,-4.1,1.9,76,...,0,0,0,0,0,1,1.0,28.0,18.0,NaN
3,2022-03-04 00:03:00,2022-03-04 00:00:00,2022,3,4,0,3,-4.1,1.9,76,...,0,0,0,0,0,1,1.0,28.0,18.0,NaN
4,2022-03-04 00:04:00,2022-03-04 00:00:00,2022,3,4,0,4,-4.2,1.9,76,...,0,0,0,0,0,1,1.0,28.0,18.0,NaN


In [86]:
x_test=df.drop(['내부 온도', '시간', '년월일', '년도', '월', '일', '시', '분'], axis=1)
y_test=df['내부 온도']

manipulated_df=pd.DataFrame([[0]*(len(x_test.columns))]*len(df), columns=x_test.columns)

manipulated_df.head()


,일사량(W/㎡),외기온(℃),외기습도(%),풍속(m/s),풍향(º),내부 습도,차광스크린(수평),차열스크린(수평),에너지스크린(수평),에너지스크린(수직),배기팬,천창,유동팬,팬코일 B동,환기온도(천창 제어온도),난방온도(설정온도),열공급량(kWh)
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [87]:
# x_test['환기온도(천창 제어온도)'] 28
# x_test['난방온도(설정온도)'] 18

In [88]:
# 모델의 예측값
yhat_list = [16, 17, 18, 19, 20, 21, 29, 30, 29, 25 ]

for i in range(1, len(x_test)):
    # 하나씩 넣어야함(매차례 변동)
    # yhat = model.predict(x_test)
    yhat=yhat_list[i]

    curr_temp = y_test[i-1]
    pred_temp = yhat
    max_thres = x_test['환기온도(천창 제어온도)'][i-1]
    min_thres = x_test['난방온도(설정온도)'][i-1]
    a, b, c, d, e, f, g, h = manipulate_minmax(curr_temp, pred_temp, max_thres, min_thres, RRS)
    if a != -1:
        x_test['배기팬'][i], x_test['유동팬'][i], x_test['천창'][i], x_test['팬코일 B동'][i], x_test['차광스크린(수평)'][i], x_test['차열스크린(수평)'][i], x_test['에너지스크린(수평)'][i], x_test['에너지스크린(수직)'][i] = a, b, c, d, e, f, g, h

    aa, bb, cc, dd, ee, ff, gg, hh = manipulate_optval(curr_temp, pred_temp)
    if aa != -1:
        x_test['배기팬'][i], x_test['유동팬'][i], x_test['천창'][i], x_test['팬코일 B동'][i], x_test['차광스크린(수평)'][i], x_test['차열스크린(수평)'][i], x_test['에너지스크린(수평)'][i], x_test['에너지스크린(수직)'][i] = aa, bb, cc, dd, ee, ff, gg, hh 

   

현재 온도: 17
예측 온도: 17
최대 온도: 28
최소 온도: 18
작동: 양의 방향
------------------------


/var/folders/h7/xtfp3kvj4b35bzvxsxgpw5wc0000gn/T/ipykernel_40456/1949176184.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test['배기팬'][i], x_test['유동팬'][i], x_test['천창'][i], x_test['팬코일 B동'][i], x_test['차광스크린(수평)'][i], x_test['차열스크린(수평)'][i], x_test['에너지스크린(수평)'][i], x_test['에너지스크린(수직)'][i]  = a, b, c, d, e, f, g, h


현재 온도: 17
예측 온도: 18
최대 온도: 28
최소 온도: 18
에러에러
현재 온도: 17
예측 온도: 19
최대 온도: 28
최소 온도: 18
작동 정지: 온도가 적정 범위 내에 있음
------------------------
현재 온도: 17
예측 온도: 20
최대 온도: 28
최소 온도: 18
작동 정지: 온도가 적정 범위 내에 있음
------------------------
현재 온도: 17
예측 온도: 21
최대 온도: 28
최소 온도: 18
작동 정지: 온도가 적정 범위 내에 있음
------------------------
현재 온도: 17
예측 온도: 29
최대 온도: 28
최소 온도: 18
작동: 음의 방향
------------------------
현재 온도: 17
예측 온도: 30
최대 온도: 28
최소 온도: 18
작동: 음의 방향
------------------------
현재 온도: 17
예측 온도: 29
최대 온도: 28
최소 온도: 18
작동: 음의 방향
------------------------
현재 온도: 17
예측 온도: 25
최대 온도: 28
최소 온도: 18
작동 정지: 온도가 적정 범위 내에 있음
------------------------


In [94]:
x_test

,일사량(W/㎡),외기온(℃),외기습도(%),풍속(m/s),풍향(º),내부 습도,차광스크린(수평),차열스크린(수평),에너지스크린(수평),에너지스크린(수직),배기팬,천창,유동팬,팬코일 B동,환기온도(천창 제어온도),난방온도(설정온도),열공급량(kWh)
0,-4.0,2.0,76,0.0,5,74.105,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
1,-4.1,2.0,76,0.4,5,73.585,0,0,0,0,0,0,0,1.0,28.0,18.0,NaN
2,-4.1,1.9,76,0.0,5,73.980,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
3,-4.1,1.9,76,0.0,5,73.690,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
4,-4.2,1.9,76,0.0,5,73.625,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
5,-4.2,1.9,76,0.0,5,73.670,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
6,-4.2,1.9,76,0.0,5,73.320,100,100,100,100,1,1,1,0.0,28.0,18.0,NaN
7,-4.1,1.9,76,0.0,5,73.140,100,100,100,100,1,1,1,0.0,28.0,18.0,NaN
8,-4.1,1.9,76,0.0,5,73.140,100,100,100,100,1,1,1,0.0,28.0,18.0,NaN
9,-4.1,1.8,76,0.0,5,72.350,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN


In [95]:
df.drop(['내부 온도', '시간', '년월일', '년도', '월', '일', '시', '분'], axis=1)[:10]

,일사량(W/㎡),외기온(℃),외기습도(%),풍속(m/s),풍향(º),내부 습도,차광스크린(수평),차열스크린(수평),에너지스크린(수평),에너지스크린(수직),배기팬,천창,유동팬,팬코일 B동,환기온도(천창 제어온도),난방온도(설정온도),열공급량(kWh)
0,-4.0,2.0,76,0.0,5,74.105,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
1,-4.1,2.0,76,0.4,5,73.585,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
2,-4.1,1.9,76,0.0,5,73.980,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
3,-4.1,1.9,76,0.0,5,73.690,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
4,-4.2,1.9,76,0.0,5,73.625,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
5,-4.2,1.9,76,0.0,5,73.670,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
6,-4.2,1.9,76,0.0,5,73.320,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
7,-4.1,1.9,76,0.0,5,73.140,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
8,-4.1,1.9,76,0.0,5,73.140,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN
9,-4.1,1.8,76,0.0,5,72.350,0,0,0,0,0,0,1,1.0,28.0,18.0,NaN


---
## 고려해야할 하이퍼파라미터
1. 차광, 차열, 에너지 스크린(수평, 수직), 천창의 열림 퍼센트
2. 목표값: 최대, 최소온도 vs 내부 온도
3. RRS: 0.5 ~ 1.5 사이 0.1씩 증감
4. 최대 온도와 최소 온도에서 가능 온도의 범위

### 1. 스크린과 천창들의 열림 퍼센트
- 현재는 0, 30, 50, 100으로 구성
- 온도 차이가 많이 날수록 열림 단계를 상승? 

### 2. 목표값: 최대, 최소 온도 vs 내부 온도
- 두 경우 모두 만들어서 열공급량이 더 적은 것 선택
- 최소, 최대 온도의 경우 목표하는 온도가 범위값이라 온도로 평가하기 어려움. 열공급량으로 평가해야함

### 3. RRS: 0.5~1.5
- RRS가 작을수록 필요 열공급량이 줄어들까?
- 아닌 것 같음. 이것도 열공급량별로 보고 결정하는 것이 좋아보임!

### 4. 최대 온도와 최소 온도 사이에서 가능 온도의 범위
- 최대, 최소 온도를 넘어섰을 때 패널티를 줘야하나?
- 최소 온도 앞 뒤로 쿠션을 줘서 여기까진 가능하다.. 이런걸 줘야하나
- 나중에 해도 될 것 같음. 전부다 예측해보고 많이 넘어가는 거 있으면 살펴보지 뭐!